# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [3]:
encoding_dim = 17
input_reg = keras.Input(shape=(vec_length,))
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(70, activation='relu')(encoded)
encoded = layers.Dense(30, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = encoded_input
decoded = layers.Dense(30, activation='relu')(decoded)
decoded = layers.Dense(70, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [4]:
opt = optimizers.Adamax(learning_rate=0.003)
autoencoder.compile(optimizer=opt, loss='binary_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=100,
                batch_size=16,
                shuffle=True,
                validation_data=(val_vecs, val_vecs))

Epoch 1/100
625/625 [==============================] - 1s 941us/step - loss: 0.1937 - val_loss: 0.1777
Epoch 2/100
625/625 [==============================] - 1s 806us/step - loss: 0.1701 - val_loss: 0.1629
Epoch 3/100
625/625 [==============================] - 1s 811us/step - loss: 0.1548 - val_loss: 0.1488
Epoch 4/100
625/625 [==============================] - 1s 811us/step - loss: 0.1437 - val_loss: 0.1418
Epoch 5/100
625/625 [==============================] - 1s 821us/step - loss: 0.1353 - val_loss: 0.1336
Epoch 6/100
625/625 [==============================] - 1s 814us/step - loss: 0.1264 - val_loss: 0.1248
Epoch 7/100
625/625 [==============================] - 1s 813us/step - loss: 0.1204 - val_loss: 0.1206
Epoch 8/100
625/625 [==============================] - 1s 827us/step - loss: 0.1166 - val_loss: 0.1171
Epoch 9/100
625/625 [==============================] - 1s 824us/step - loss: 0.1124 - val_loss: 0.1142
Epoch 10/100
625/625 [==============================] - 1s 828us/step - l

Epoch 80/100
625/625 [==============================] - 1s 812us/step - loss: 0.0049 - val_loss: 0.0075
Epoch 81/100
625/625 [==============================] - 0s 789us/step - loss: 0.0041 - val_loss: 0.0055
Epoch 82/100
625/625 [==============================] - 1s 814us/step - loss: 0.0041 - val_loss: 0.0047
Epoch 83/100
625/625 [==============================] - 0s 797us/step - loss: 0.0040 - val_loss: 0.0033
Epoch 84/100
625/625 [==============================] - 0s 782us/step - loss: 0.0032 - val_loss: 0.0047
Epoch 85/100
625/625 [==============================] - 0s 785us/step - loss: 0.0031 - val_loss: 0.0049
Epoch 86/100
625/625 [==============================] - 1s 804us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 87/100
625/625 [==============================] - 1s 821us/step - loss: 0.0024 - val_loss: 0.0043
Epoch 88/100
625/625 [==============================] - 1s 828us/step - loss: 0.0028 - val_loss: 0.0027
Epoch 89/100
625/625 [==============================] - 1s 805us

Evaluation

In [5]:
decoded_regs = autoencoder.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
overall accuracy = 0.998
by digit accuracy:
    1.0
    1.0
    1.0
    1.0
    1.0
    0.998
    1.0
